In [1]:
import numpy as np
import pandas as pd 
import networkx as nx
from graphviz import Digraph
import matplotlib.pyplot as plt

In [2]:
df_alpha = pd.read_csv('./soc-sign-bitcoinalpha.csv',header=None)
df_alpha.rename(columns = {0:'SOURCE', 1:'TARGET',2:'RATING',3:'TIME'}, inplace = True)
df_alpha["RATING"] = df_alpha["RATING"].map(lambda x:x/10)
df_alpha

,SOURCE,TARGET,RATING,TIME
0,7188,1,1.0,1407470400
1,430,1,1.0,1376539200
2,3134,1,1.0,1369713600
3,3026,1,1.0,1350014400
4,3010,1,1.0,1347854400
...,...,...,...,...
24181,7604,7601,1.0,1364270400
24182,7601,7604,1.0,1364270400
24183,7604,7602,1.0,1364270400
24184,7602,7604,1.0,1364270400


In [3]:
fairness = dict()
for i in df_alpha.index:
    fairness[df_alpha.loc[i, "SOURCE"]] = 1

goodness = dict()
for i in df_alpha.index:
    goodness[df_alpha.loc[i, "TARGET"]] = 1

In [4]:
def next_goodness(g: dict, f: dict):
    for j, v in enumerate(g.keys()):
        sum = 0
        df_v = df_alpha[df_alpha.loc[:, "TARGET"] == v]
        in_v = len(df_v)
        for i in df_v.index:
            sum += f[df_v.loc[i, "SOURCE"]] * df_v.loc[i, "RATING"]

        g[v] = sum / in_v
    return g

def next_fairness(f: dict, g: dict):
    for u in f.keys():
        sum = 0
        df_u = df_alpha[df_alpha.loc[:, "SOURCE"] == u]
        out_u = len(df_u)
        for i in df_u.index:
            sum += abs(df_u.loc[i, "RATING"] - g[df_u.loc[i, "TARGET"]]) / 2
        f[u] = 1 - sum / out_u
    return f

def judge_goodness(good:dict, next:dict):
    sum = 0
    for u, g in good.items():
        g_next = next[u]
        sum += abs(g - g_next)
    if(sum > 0.001):
        return True
    else:
        return False

def judge_fairness(fair: dict, next: dict):
    sum = 0
    for u, f in fair.items():
        f_next = next[u]
        sum += abs(f - f_next)
    if(sum > 0.001):

        return True
    else:
        return False
import time
def FGA(dict_f: dict, dict_g: dict):
    t = 0
    next_g = next_goodness(dict_g.copy(), dict_f.copy())
    next_f = next_fairness(dict_f.copy(), dict_g.copy())
    s1 = time.time()
    
    while (judge_goodness(dict_g, next_g) or judge_fairness(dict_f, next_f)):
        print(t)
        t = t + 1
        dict_g = next_g.copy()
        dict_f = next_f.copy()

        print("judge:", time.time() - s1)
        s1 = time.time()

        next_g = next_goodness(next_g.copy(), next_f.copy())
        next_f = next_fairness(next_f.copy(), next_g.copy())
        print("next:", time.time() - s1)
        s1 = time.time()
        
    return next_g, next_f, t

In [5]:
good, fair, t = FGA(fairness, goodness)

0
judge: 0.0012090206146240234
next: 1.6246919631958008
1
judge: 0.0008380413055419922
next: 1.617386817932129
2
judge: 0.0007889270782470703
next: 1.6376876831054688
3
judge: 0.0008890628814697266
next: 1.6031222343444824
4
judge: 0.0007562637329101562
next: 1.6428842544555664
5
judge: 0.000743865966796875
next: 1.6266696453094482
6
judge: 0.0011241436004638672
next: 1.6773920059204102
7
judge: 0.0007240772247314453
next: 1.59425687789917
8
judge: 0.0007767677307128906
next: 1.6312909126281738
9
judge: 0.0008027553558349609
next: 1.6506669521331787
10
judge: 0.0008280277252197266
next: 1.613023042678833
11
judge: 0.0007381439208984375
next: 1.5968191623687744


In [6]:
import pickle
fairness_of_nodes = open("./fairness_of_nodes.pickle", "wb")
pickle.dump(fair, fairness_of_nodes)
fairness_of_nodes.close()
goodness_of_nodes = open("./goodness_of_nodes.pickle", "wb")
pickle.dump(good, goodness_of_nodes)
goodness_of_nodes.close()

In [7]:
max(fair.values()), min(fair.values())

(0.9999999999999999, 0.35135923922144374)

In [8]:
max(good.values()), min(good.values())

(0.9937442766341511, -0.99951171875)